In [1]:
from typing import Dict, List, Tuple
import torch
import numpy as np
import gym
import json
from plotly.subplots import make_subplots
from DQN_Agent import *
from Logger import *


%load_ext autoreload
%autoreload 2

## Set environment and agent parameters

In [2]:
env_arguments = {
"alpha":1,
"beta":1,
"min_temp" : 16,
"max_temp" : 21,
"action_dim": 100,
"modelname":'CELLS_v1.fmu',
"simulation_path":  r'C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\custom_gym\Eplus_simulation',
"param_list": ['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
"days": 151,  
"hours": 24,  
"minutes": 60,
"seconds": 60,
"ep_timestep": 6
}

agent_arguments = {
"memory_size": [1000],
"batch_size": [32],
"target_update": [100],
"epsilon_decay": [1 / 20000],
"max_epsilon": [1],
"min_epsilon":  [0.0],
"gamma": [0.99],
"inside_dim": [128,256],
"num_hidden_layers": [1],
"seed": [778]
}

In [3]:
from utils import all_combinations_list

num_episodes = 1
num_iterations = 300

for curr_agent_arguments in all_combinations_list(agent_arguments):

    env = gym.make('EnergyPlusEnv-v0')
    env.set_arguments(env_arguments)

    agent = DQNAgent(env,dict_arguments=curr_agent_arguments)
    total_cum_reward, total_cum_heat, results_path = agent.train(num_episodes=num_episodes, num_iterations=num_iterations ,log=False)



Agent Action_dim100
Env action_dim 100
 BETA 1
Iteration0
Iteration100
Iteration200
(226.1573406953302, 31261869.974988677, 'c:\\Users\\Harold\\Desktop\\ENAC-Semester-Project\\DIET_Controller/results/2022_4_29/results_2022_4_29_18_4')
Agent Action_dim100
Env action_dim 100
 BETA 1
Iteration0
Iteration100
Iteration200
(226.14408752263398, 31262763.844984073, 'C:\\Users\\Harold\\Desktop\\ENAC-Semester-Project\\DIET_Controller\\custom_gym\\Eplus_simulation/results/2022_4_29/results_2022_4_29_18_4')


In [38]:
# Example of how to use the function: 

from utils import search_paths

searching_directory = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\results\2022_4_28"

conditions={
    "num_iterations": ["=",21744] # to only have have trainings where the full simulation was used
}

conditions["pmv"] = {
        "[-inf,-2]": [">",0.2]
}

path_list = search_paths(searching_directory,conditions)

KeyError: 'num_iterations'